In [ ]:

#更正之新代码
import dash
import pandas as pd
import plotly.express as px
from dash import dcc, html
from dash.dependencies import Input, Output, State

# Load the data using pandas
data = pd.read_csv('/Users/lirui/Desktop/硕博/文凭材料/文憑/IBM-DATA-ANALYST-DATA/Data Visualization with Python/Week '
                   '5/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("汽车销售统计仪表板", style={'textAlign': 'center', 'color': '#503D36', 'fontSize': 24}),
    html.Div([
        html.Label("选择统计类型："),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[
                {'label': '年度统计', 'value': 'Yearly Statistics'},
                {'label': '衰退期统计', 'value': 'Recession Period Statistics'}
            ],
            placeholder='选择报告类型'
        ),
        html.Label("选择年份："),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': str(year), 'value': year} for year in data['Year'].unique()],
            placeholder='选择年份',
            disabled=True
        ),
        html.Button("显示图表", id='show-chart-button'),
    ]),
    html.Div(id='output-container')
])

@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value'))
def update_input_container(selected_statistics):
    return selected_statistics != '年度统计'

@app.callback(
    Output('output-container', 'children'),
    Input('show-chart-button', 'n_clicks'),
    [State('dropdown-statistics', 'value'),
     State('select-year', 'value')])
def update_output_container(n_clicks, selected_statistics, input_year):
    if not n_clicks:
        return []

    if selected_statistics == '年度统计':
        yearly_data = data[data['Year'] == input_year]
        
        sales_by_year = yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig1 = dcc.Graph(figure=px.line(sales_by_year, x='Year', y='Automobile_Sales', title="年度汽车销售额"))

        sales_by_type = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig2 = dcc.Graph(figure=px.line(sales_by_type, x='Vehicle_Type', y='Automobile_Sales', title="每月汽车销售额"))

        return html.Div([
            html.Div([fig1], className='chart-item'),
            html.Div([fig2], className='chart-item')
        ])

    elif selected_statistics == '衰退期统计':
        recession_data = data[data['Recession'] == 1]

        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig1 = dcc.Graph(figure=px.line(yearly_rec, x='Year', y='Automobile_Sales', title="衰退期平均汽车销售额"))

        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig2 = dcc.Graph(figure=px.line(average_sales, x='Vehicle_Type', y='Automobile_Sales', title="不同车型平均销售额"))

        # 图表 3：使用饼图显示不同车型的总广告支出份额
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        fig3 = dcc.Graph(figure=px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type', title="衰退期不同车型的广告支出份额"))

        # 图表 4：使用柱状图显示失业率对不同车型和销售额的影响
        unemployment_by_type = recession_data.groupby('Vehicle_Type')['unemployment_rate'].mean().reset_index()
        fig4 = dcc.Graph(figure=px.bar(unemployment_by_type, x='Vehicle_Type', y='unemployment_rate', title="失业率对不同车型和销售额的影响"))

        return html.Div([
            html.Div([fig1], className='chart-item'),
            html.Div([fig2], className='chart-item'),
            html.Div([fig3], className='chart-item'),
            html.Div([fig4], className='chart-item')
        ])

if __name__ == '__main__':
    app.run_server(debug=True)

    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
# --------------------------------------------------------------------------------
#旧代码
#!/usr/bin/env python
# coding: utf-8

# In[ ]:
import dash
import pandas as pd
import plotly.express as px
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

# Load the data using pandas
data = pd.read_csv('/Users/lirui/Desktop/硕博/文凭材料/文憑/IBM-DATA-ANALYST-DATA/Data Visualization with Python/Week '
                   '5/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
# app.title = "Automobile Statistics Dashboard"

# ---------------------------------------------------------------------------------
# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]
# List of years
year_list = [i for i in range(1980, 2024, 1)]
# ---------------------------------------------------------------------------------------
# Create the layout of the app
app.layout = html.Div([
    # TASK 2.1 Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard", style={'textAlign': 'center','color': '#503D36','fontSize':24}) ,# May include style for title
        html.Div([  # TASK 2.2: Add two dropdown menus
            html.Label("Select Statistics:"),
            dcc.Dropdown(id='dropdown-statistics',
                    options=[
                        {'label':'Yearly Statistics','value':'Yearly statistics'},
                        {'label':'Recession Period statistics','value':'Recession Period statistics'}
                    ],
                    placeholder='Select a report type'),
            html.Label("Select Year:"),
            dcc.Dropdown(
                id='select-year',
                options=[{'label': i, 'value': i} for i in year_list],
                placeholder='Select-year',
                disabled=True
            )
        ]),
        html.Div(id='output-container', className='chart-grid', style={'display':'flex'})
])

# TASK 2.4: Creating Callbacks
    # Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value'))
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False


# Callback for plotting
    # Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value')
    ,Input(component_id='select-year', component_property='value')])

def update_output_container(selected_statistics, input_year, data=None, average_sales=None):
    if selected_statistics == 'Yearly Statistics':
        return dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            placeholder='Select-year'
        )

        recession_data = data[data['Recession'] == 1]
        # TASK 2.5: Create and display graphs for Recession Report Statistics

        # Plot 1 Automobile sales fluctuate over Recession Period (year wise)
            # use groupby to create relevant data for plotting
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
                figure=px.line(yearly_rec,
                    x='Year',
                    y='Automobile_Sales',
                    title="Average Automobile Sales fluctuation over Recession Period"))

        # Plot 2 Calculate the average number of vehicles sold by vehicle type
            # use groupby to create relevant data for plotting
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].reset_index()
        R_chart2 = dcc.Graph(
            figure=px.line(average_sales,
                x='Vehicle_Type',
                y='Automobile_Sales',
                title="Average Number of Vehicles Sold by Vehicle Tpye"))

        # Plot 3 Pie chart for total expenditure share by vehicle type during recessions
            # use groupby to create relevant data for plotting
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec,
            values='Expenditure',
            names='Vehicle Type',
            title="Total Expenditure Share by vehicle Type"
        )
    )
        # Plot 4 bar chart for the effect of unemployment rate on vehicle type and sales
        data = recession_data.groupby('vehicle_Type')['unemployment_rate'].count().reset_index()
        R_chart4 = dcc.Graph(
                figure=px.bar(data,
                values='Count',
                names='Unemployment Rate',
                title="Effect of Unemployment Rate on vehicle Type and Sales"
        )
    )


    # TASK 2.6: Create and display graphs for Yearly Report Statistics
    # Yearly Statistic Report Plots
    elif (input_year and selected_statistics == 'Recession Period Statistics'):
        yearly_data = data[data['Year'] == 2011]

        # TASK 2.5: Creating Graphs Yearly data

        # plot 1 Yearly Automobile sales using line chart for the whole period.
        sales_by_year = yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig1 = px.line(sales_by_year,
                x='Year',
                y='Automobile_Sales',
                title="Average Automobile Sales fluctuation over Recession Period")

        # Plot 2 Total Monthly Automobile sales using line chart.
        sales_by_type = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig2 = px.line(sales_by_type,
                x='Vehicle_Type',
                y='Automobile_Sales',
                title="Average Number of vehicles Sold by Vehicle Type")

        # Plot bar chart for average number of vehicles sold during the given year
        sales_by_year = yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig3 = px.bar(sales_by_year,
                x='Year',
                y='Automobile_Sales',
                title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year))

        # Total Advertisement Expenditure for each vehicle using pie chart
        exp_by_type = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        fig4 = px.pie(exp_by_type,
                names='Vehicle_Type',
                values='Advertising_Expenditure',
                title = "Total Expenditure Share by Vehicle Type")

        # TASK 2.6: Returning the graphs for displaying Yearly data
        return [
            html.Div(className='chart-item', children=[
                html.Div([fig1],id='plot1'),
                html.Div([fig2], id='plot2'),
                html.Div([fig3], id='plot3'),
                html.Div([fig4], id='plot4'),
            ], style={'display':'flex','flex-wrap':'wrap'}),
        ]

    else:
        return None


# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)